In [50]:
import os
import pandas as pd
from tqdm import tqdm
import shutil
from src import utils
from scipy.stats import spearmanr
import math

In [51]:
# Project path
project_dir = os.path.abspath('')[:-9]
# Load enviromental variables

env_var = utils.load_env_variables(project_dir)
env_var["root_path"] = "/home/tpinho/IJGIS/Datasets/NSW_Election_2019"

In [52]:
dataset = "Original"
scv_methods = ["CrossValidation", "Optimistic", "RegGBSCV_R_Kappa_0.0", "TraditionalSCV"]
#scv_methods = ["RegGBSCV_R_Kappa_0.0", "RegGBSCV_R_Kappa_0.1", 
#           "RegGBSCV_R_Kappa_0.2", "RegGBSCV_R_Kappa_0.3", 
#           "RegGBSCV_R_Kappa_0.4", "RegGBSCV_R_Kappa_0.5"]
fs_method = "CFS"
metric = "SP_INV"

In [53]:
from pandas import isna


path = os.path.join(env_var["root_path"], dataset, "results")
ml_methods = ["KNN", "OLS", "Lasso", "Ridge", "ElasticNet", "DT", "LGBM", "RF", "MLP", "SVM"]
results = pd.DataFrame(index=ml_methods, columns=scv_methods)
for scv_method in tqdm(scv_methods):
    for ml_method in ml_methods:
        metrics = pd.read_csv(os.path.join(path, scv_method, "evaluations", fs_method, ml_method, "metrics_with_mcpm.csv"))
        metrics = metrics[metrics["FOLD"] != 53 ]
        metrics = metrics[metrics["TEST_SIZE"]>10]
        results.loc[ml_method, scv_method] = metrics[metric].mean()
        if metrics.isnull().values.any():
            print(f"{scv_method}: {ml_method}")

results.index.names = ["Methods"]
results.to_csv(os.path.join(env_var["root_path"], dataset, "comparison", f"{fs_method}_{metric}.csv"))


100%|██████████| 4/4 [00:00<00:00, 20.68it/s]
